In [1]:
!pip install langchain

In [2]:
!pip install openai

In [3]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-" #openai 키 입력

llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 
                 model_name='gpt-4',  # 모델명
                )

# 프롬프트 생성
prompt = PromptTemplate(
  input_variables=["country"],
  template= "{country}의 수도는 어디야?",
)

chain = LLMChain(llm=llm, prompt=prompt) #프롬프트와 모델을 체인으로 연결 
chain.run("대한민국")

/opt/anaconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/opt/anaconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/anaconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'대한민국의 수도는 서울입니다.'

In [4]:
# chain 객체의 run 메소드를 호출하고 "대한민국"을 인자로 전달.
# 프롬프트를 통해 "{대한민국}의 수도는 어디야?"라는 질문을 생성하고, 설정된 언어 모델을 사용하여 이 질문에 대한 답변을 생성.

In [6]:
#프롬프트1 정의
prompt1 = PromptTemplate(
    input_variables=['sentence'],
    template="다음 문장을 한글로 번역하세요.\n\n{sentence}"
)
#번역(체인1)에 대한 모델
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="translation")

#프롬프트2 정의
prompt2 = PromptTemplate.from_template(
    "다음 문장을 한 문장으로 요약하세요.\n\n{translation}"
)
#요약(체인2)에 대한 모델
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="summary")

# SequentialChain을 사용하여 체인 두개(번역, 요약)를 연결
from langchain.chains import SequentialChain
all_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=['sentence'],
    output_variables=['translation','summary'],
)
sentence="""
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""
all_chain(sentence)

{'sentence': '\nOne limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.\nFor this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.\n',
 'translation': 'LLM의 한 가지 제한점은 문맥 정보(예: 특정 문서나 이메일에 대한 접근)의 부족입니다. 이를 극복하기 위해 LLM에게 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다. 이를 위해 먼저 문서 로더를 사용하여 외부 데이터를 로드해야 합니다. LangChain은 PDF와 이메일부터 웹사이트와 유튜브 비디오에 이르기까지 다양한 유형의 문서에 대한 로더를 제공합니다.',
 'summary': 'LLM의 문맥 정보 부족이라는 제한점을 극복하기 위해, 외부 데이터에 접근할 수 있도록 권한을 부여하고, LangChain의 문서 로더를 통해 PDF, 이메일, 웹사이트, 유튜브 비디오 등 다양한 유형의 문서를 로드할 수 있습니다.'}

In [7]:
# 체인을 통해서 프롬프트와 모델, 모델과 모델을 연결하는 등 여러 구성 요소를 조합하여 하나의 파이프라인을 구성할 수 있음.
# 위 예시에서는 영어로 이루어진 문장을 한글로 번역한 후 (체인1) 그 문장을 다시 한 문장으로 요약하는 (체인2) 방식으로 구성하였음.